 I'd need a proper data set to do a fair comparison
 but i couldnt find a simple data set which was good for
 naive bayes within a few google searches..

In [15]:
from numpy.random import randn
from numpy import array, concatenate, zeros, ones

cluster0 = zeros((10000, 10))
for index in range(10000):
    cluster0[index] = randn(10) + 20

cluster1 = zeros((10000, 10))
for index in range(10000):
    cluster1[index] = randn(10) - 20
    

train0 = cluster0[:8000]
train1 = cluster1[:8000]

test0 = cluster0[8000:]
test1 = cluster1[8000:]

train = concatenate([train0, train1])
test = concatenate([test0, test1])

train_labels = concatenate([zeros(8000), ones(8000)])
test_labels = concatenate([zeros(2000), ones(2000)])

TEST_DATA = 4000

In [16]:
from naive_bayes import naive_bayesian
from bayes import bayesian
from time import time
from numpy import arange

notnaive = bayesian()
naive = naive_bayesian()

timestamp = time()
notnaive.train(train, train_labels)
print("Time to train notnaive: {}".format(time() - timestamp))

timestamp = time()
naive.train(train, train_labels)
print("Time to train naive: {}".format(time() - timestamp))

timestamp = time()
correct = 0
for index, t in enumerate(test):
    correct += (naive.predict(t) == test_labels[index])

print("Naive Stats")
print("Correct rate: {}".format(correct / TEST_DATA))
print("Avarage predict rate: {}".format((time() - timestamp) / TEST_DATA))

print("\n")

timestamp = time()
correct = 0
for index, t in enumerate(test):
    correct += (notnaive.predict(t) == test_labels[index])
    
print("NotNaive Stats")
print("Correct rate: {}".format(correct / TEST_DATA))
print("Avarage predict rate: {}".format((time() - timestamp) / TEST_DATA))

Time to train notnaive: 0.003715038299560547
Time to train naive: 0.0032722949981689453
Naive Stats
Correct rate: 1.0
Avarage predict rate: 5.887699127197266e-05


NotNaive Stats
Correct rate: 1.0
Avarage predict rate: 7.596904039382934e-05
